# Carregando resultados das execuções para posteriores testes estatísticos 

In [1]:
import pandas as pd
from scipy.stats import fisher_exact

def carregar_resultados():
  todos_os_resultados = pd.read_excel('resultados_finais-estatistica.xlsx', sheet_name='Resultados brutos')
  abordagens = todos_os_resultados['ABORDAGEM'].unique()
  resultados_dm = todos_os_resultados[todos_os_resultados['MEDIDA'] == 'DM']
  resultados_rm = todos_os_resultados[todos_os_resultados['MEDIDA'] == 'RM']
  return abordagens, resultados_dm, resultados_rm

def produzir_tabelas_de_contingencia(resultados, tratamento1, tratamento2, separar_por_banco = False):
  coluna_resultado = 'RESULTADO'
  coluna_abordagem = 'ABORDAGEM'
  coluna_banco = 'BANCO'
  indice_colunas = [coluna_resultado]
  indice_linhas = [coluna_abordagem]
  if separar_por_banco:
    indice_linhas.append(coluna_banco)
    indice_linhas = indice_linhas[::-1]
  alvo = resultados[resultados[coluna_abordagem].isin([tratamento1,tratamento2])][indice_colunas+indice_linhas]
  tabela = pd.pivot_table(alvo, index=indice_linhas, columns=indice_colunas, aggfunc=len, fill_value=0)
  tabela.columns = [str(x) for x in tabela.columns]
  tabela = tabela[['True', 'False']]
  return tabela
abordagens, resultados_dm, resultados_rm = carregar_resultados()

# Executando teste estatístico aos pares considerando medida DM

Saída corresponde a um CSV com as seguintes colunas:
- Banco: 0 indicando todos
- Abordagem 1
- Abordagem 2
- p-value
- Desempenho da abordagem 1
- Desempenho da abordagem 2

In [2]:
for b in range(24):
  for i in range(len(abordagens)-1):
    for j in range(i+1,len(abordagens)):
      abordagem1 = abordagens[i]
      abordagem2 = abordagens[j]
      contingencia = produzir_tabelas_de_contingencia(resultados_dm, abordagem1, abordagem2, b > 0)
      if b > 0:
        contingencia = contingencia.loc[b]
      _, pvalue = fisher_exact(contingencia)

      s = contingencia.loc[abordagem1]
      p1 = s['True'] / sum(s)
      
      s = contingencia.loc[abordagem2]
      p2 = s['True'] / sum(s)

      print(b, abordagem1, abordagem2, "{:.2f}".format(pvalue*100), p1, p2, sep=',')


0,Seminal,Abordagem 1.1,0.13,0.16956521739130434,0.09565217391304348
0,Seminal,Abordagem 1.2,60.45,0.16956521739130434,0.18478260869565216
0,Seminal,Abordagem 1.3,26.94,0.16956521739130434,0.2
0,Seminal,Abordagem 1.4,20.46,0.16956521739130434,0.20434782608695654
0,Seminal,Abordagem 2,17.69,0.16956521739130434,0.20652173913043478
0,Seminal,Abordagem 3.1,0.00,0.16956521739130434,0.7391304347826086
0,Seminal,Abordagem 3.2,0.00,0.16956521739130434,0.8782608695652174
0,Abordagem 1.1,Abordagem 1.2,0.01,0.09565217391304348,0.18478260869565216
0,Abordagem 1.1,Abordagem 1.3,0.00,0.09565217391304348,0.2
0,Abordagem 1.1,Abordagem 1.4,0.00,0.09565217391304348,0.20434782608695654
0,Abordagem 1.1,Abordagem 2,0.00,0.09565217391304348,0.20652173913043478
0,Abordagem 1.1,Abordagem 3.1,0.00,0.09565217391304348,0.7391304347826086
0,Abordagem 1.1,Abordagem 3.2,0.00,0.09565217391304348,0.8782608695652174
0,Abordagem 1.2,Abordagem 1.3,61.59,0.18478260869565216,0.2
0,Abordagem 1.2,Abordagem 1.4,50.53,0.18478

# Executando teste estatístico aos pares considerando medida RM

Saída também é um CSV

In [3]:
for b in range(24):
  for i in range(len(abordagens)-1):
    for j in range(i+1,len(abordagens)):
      abordagem1 = abordagens[i]
      abordagem2 = abordagens[j]
      contingencia = produzir_tabelas_de_contingencia(resultados_rm, abordagem1, abordagem2, b > 0)
      if b > 0:
        contingencia = contingencia.loc[b]
      _, pvalue = fisher_exact(contingencia)

      s = contingencia.loc[abordagem1]
      p1 = s['True'] / sum(s)
      
      s = contingencia.loc[abordagem2]
      p2 = s['True'] / sum(s)

      print(b, abordagem1, abordagem2, "{:.2f}".format(pvalue*100), p1, p2, sep=',')


0,Seminal,Abordagem 1.1,0.31,0.21304347826086956,0.13695652173913042
0,Seminal,Abordagem 1.2,100.00,0.21304347826086956,0.21304347826086956
0,Seminal,Abordagem 1.3,30.72,0.21304347826086956,0.24347826086956523
0,Seminal,Abordagem 1.4,24.02,0.21304347826086956,0.24782608695652175
0,Seminal,Abordagem 2,21.10,0.21304347826086956,0.25
0,Seminal,Abordagem 3.1,0.00,0.21304347826086956,0.808695652173913
0,Seminal,Abordagem 3.2,0.00,0.21304347826086956,0.9565217391304348
0,Abordagem 1.1,Abordagem 1.2,0.31,0.13695652173913042,0.21304347826086956
0,Abordagem 1.1,Abordagem 1.3,0.01,0.13695652173913042,0.24347826086956523
0,Abordagem 1.1,Abordagem 1.4,0.00,0.13695652173913042,0.24782608695652175
0,Abordagem 1.1,Abordagem 2,0.00,0.13695652173913042,0.25
0,Abordagem 1.1,Abordagem 3.1,0.00,0.13695652173913042,0.808695652173913
0,Abordagem 1.1,Abordagem 3.2,0.00,0.13695652173913042,0.9565217391304348
0,Abordagem 1.2,Abordagem 1.3,30.72,0.21304347826086956,0.24347826086956523
0,Abordagem 1.2,Abordagem 

# Gerando síntese de análises

Saídas dos testes anteriores estão tabuladas no arquivo *resultados_finais-estatistica.xlsx*

Como saída, vai gerar:
- TODOS.png: arquivo com as diferenças estatísticas em todos os bancos juntos considerando DM e RM
- ALL_DM.png: arquivo com as diferenças estatísticas em cada banco separadamente considerando DM
- ALL_RM.png: arquivo com as diferenças estatísticas em cada banco separadamente considerando RM
- conclusoes.zip: arquivo compactado com as imagens de cada banco separadas

In [2]:
from zipfile import ZipFile
import sys
from PIL import Image, ImageChops
import glob
import os
import graphviz

estatisticas = pd.read_excel('resultados_finais-estatistica.xlsx', sheet_name='Estatística detalhada')
desempenhos = pd.read_excel('resultados_finais-estatistica.xlsx', sheet_name='Desempenhos detalhados')
bancos_considerados = ['TODOS'] + list(range(1,24))
metricas = ['RM', 'DM']
imagens = []
for b in bancos_considerados:
  for m in metricas:
    resumo = ''
    e = estatisticas[(estatisticas['MEDIDA'] == m) & (estatisticas['BANCO'] == b)]
    d = desempenhos[((desempenhos['MEDIDA'] == m) & (desempenhos['Banco'] == b))]
    d = d.drop(['MEDIDA', 'Banco'], axis = 1)
    d = d.transpose()
    d.columns = ['DESEMPENHO']
    d = d.sort_values(by='DESEMPENHO')
    for i in range(len(d)-1):
      j = i + 1
      a1 = d.index[i]
      a2 = d.index[j]
      d1 = float(d.iloc[i]['DESEMPENHO'])
      d2 = float(d.iloc[j]['DESEMPENHO'])
      p = float(e[(e['ABORDAGEM 1'] == a1) & (e['ABORDAGEM 2'] == a2) | (e['ABORDAGEM 1'] == a2) & (e['ABORDAGEM 2'] == a1)]['P'])
      relacao = '=' if d2 == d1 else '<'
      relacao += 'D' if p < 5 else 'i'
      if resumo == '':
        resumo = a1 + ' ' + relacao + ' ' + a2
      else:
        resumo += ' ' + relacao + ' ' + a2
    result_str = ",".join((m, str(b), resumo))
    result_str = result_str.replace('Abordagem ', '')
    result_str = result_str.replace('=i', '=')
    result_str = result_str.replace('<i', '<')
    result_str = result_str.replace(' <D ', ',')
    aux = result_str.split(',')
    label = 'Medida: ' + aux[0] + " - Banco" + ('' if aux[1].isdigit() else 's') + ": " + aux[1]
    image_name = "BD" + aux[1] + "_" + aux[0]
    imagens.append(image_name)
    dot = graphviz.Digraph(comment=image_name, format='png', graph_attr={"label": label, "splines": "ortho", 'rankdir':"LR","labeljust":"l"})
    index = 2
    while True:
      dot.node(chr(63+index),aux[index])
      if index > 2:
        dot.edge(chr(63+index-1), chr(63+index))
      index += 1
      if index >= len(aux):
        break
    dot.render(image_name,cleanup=True)  

zipObj = ZipFile('conclusoes.zip', 'w')
for im in imagens:
  zipObj.write(im + '.png')
zipObj.close()

w_offset = 800
h_offset = 89
new_im = Image.new('RGB', (w_offset*2, h_offset*12), (255, 255, 255))
new_im.paste(Image.open("BDTODOS_DM.png"), (0,0))
new_im.paste(Image.open("BDTODOS_RM.png"), (w_offset,0))
new_im=new_im.crop(ImageChops.invert(new_im).getbbox())
new_im.save("TODOS.png")

bds = list(range(1,24))
for m in ['RM','DM']:
  new_im = Image.new('RGB', (w_offset*2, h_offset*12), (255, 255, 255))
  x, y = 0, 0
  for bd in bds:
    im = Image.open("BD"+str(bd)+"_"+m+".png")
    new_im.paste(im, (x,y))
    x = w_offset if x == 0 else 0
    if x == 0:
      y += h_offset
  
  imb = ImageChops.invert(new_im).getbbox()
  new_im=new_im.crop(imb)
  new_im.save('ALL_' + m + ".png")
for f in glob.glob("BD*M.png"):
  os.remove(f)
